In [ ]:
import csv
import mysql.connector
from datetime import datetime

In [ ]:
# connecting to the mysql database on localhost using mysql.connector
mydb = mysql.connector.connect(
    host="localhost",
    database="moviehub",
    user="root",
    password="mysqlpassword"
)
mycursor = mydb.cursor()

In [ ]:
# Queries to insert data from different csv files into the database
# Queries to insert imdb data into the database
imovie_Query = "INSERT INTO IMDBMovie(primary_title, original_title, release_date, runtime, is_adult, imdb_link, genres) VALUES (%s, %s, %s, %s, %s, %s, %s)"
iperson_Query = "INSERT INTO IMDBPerson(id, name, birthday, deathday, primary_profession, known_for, imdb_link) VALUES (%s, %s, %s, %s, %s, %s, %s)"
irating_Query = "INSERT INTO IMDBRatings(id,rating,votes) VALUES (%s, %s, %s)"

# Queries to insert tmdb data into the database
tmovie_Query = "INSERT INTO TMDBMovie(title, original_title, release_date, runtime, is_adult, imdb_link, overview, popularity, poster_path, collection_belongs_to, revenue, budget, tagline, homepage_link) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
tperson_Query = "INSERT INTO TMDBPerson(id, name, birthday, deathday, alias_name, gender, biography, profile_path, popularity, place_of_birth, homepage, home_indexry, imdb_link) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

# Queries to insert rotten tomatoes data into the database
rtmovie_query = "INSERT INTO RTMovie(title, cast1, cast2, description, director, genre, rating) VALUES (%s, %s, %s, %s, %s, %s, %s)"

# Query to insert mapping data using movielens, imdb and tmdb data into the database
dbmapping_Query = "INSERT INTO Mapping(movieid,imDBId,tmdbId) VALUES (%s, %s, %s)"

In [ ]:
# inserting imdb movies data in imdbmovie table
index = 0
with open('data/imdb/title.basics.tsv', 'r',encoding='utf8') as f:
    fopen = csv.reader(f, delimiter='\t')
    for rdata in fopen:
        if index == 0:
            index += 1
            continue
        # taking only 50000 movies from the imdb movies dataset
        if index >= 304645:
            break

        index += 1
        # Extracting data from the csv file
        id = rdata[0]
        primaryTitle = rdata[2]
        originalTitle = rdata[3]
        isAdult = rdata[4]
        realeaseDate = rdata[5]
        runtime = rdata[7]
        genres = rdata[8]
        
        # converting release date to datetime object for storing in database
        if realeaseDate != '\\N':
            realeaseDate = datetime.strptime(realeaseDate, '%Y')
        else:
            realeaseDate = datetime.strptime("2000", '%Y')
            
        # converting runtime to int object for storing in database
        if runtime != '\\N':
            runtime = int(runtime)
        else:
            runtime = None
            
        # taking data from index 254645 to 304645 from the imdb dataset
        if index > 254645:
            try:
                dtuple = (primaryTitle, originalTitle, realeaseDate, runtime, isAdult, id, genres)
                mycursor.execute(imovie_Query, dtuple)
                mydb.commit()
            except Exception as exc:
                print("For Index: ",index, "error is: ",exc)

print("Data added to the database successfully!")

In [ ]:
# inserting imdb person data in imdbperson table
index = 0
with open('data/imdb/name.basics.tsv','r',encoding='utf8') as f:
    fopen = csv.reader(f, delimiter='\t')
    for rdata in fopen:
        if index == 0:
            index += 1
            continue
        # taking only 50000 movies from the imdb person dataset
        if index >= 50000:
            break
        index += 1
        # Extracting data from the csv file
        imdbLink = rdata[0]
        name = rdata[1]
        birthdate = rdata[2]
        deathdate = rdata[3]
        profession = rdata[4]
        knownFor = rdata[5]
        
        #converting birthday and deathday to none if they are not present 
        if birthdate=='' or birthdate=='\\N':
            birthdate=None
        if deathdate=='' or deathdate=='\\N':
            deathdate=None
        try:
            i = 0
            dtuple = (i,name, birthdate, deathdate, profession, knownFor, imdbLink)
            mycursor.execute(iperson_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("For Index: ",index, "error is: ",exc)

print("Data added to the database successfully!")

In [ ]:
# inserting imdb movie ratings data in imdbratings table
index = 0
with open('data/imdb/title.ratings.tsv', 'r',encoding="utf8") as f:
    fopen = csv.reader(f, delimiter='\t')

    for rdata in fopen:
        if index == 0:
            index += 1
            continue
        index += 1
        # Extracting data from the csv file
        id = rdata[0]
        rating = rdata[1]
        votes = rdata[2]
        try:
            dtuple = (id, rating, votes)
            mycursor.execute(irating_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("For Index: ",index, "error is: ",exc)

print("Data added to the database successfully!")

In [ ]:
# inserting tmdb movies data in tmdbmovie table
index = 0
with open('data/tmdb/movie.tsv', 'r',encoding="utf8") as f:
    fopen = csv.reader(f, delimiter='\t')
    
    for rdata in fopen:
        if index == 0:
            index += 1
            continue
        with open("data/ids.txt", "r") as file:
            ids = eval(file.readline())

        if int(rdata[1]) not in ids:
            continue

        index+=1
        # Extracting data from the csv file
        mvTitle = rdata[2]
        budget = rdata[5]
        homepage = rdata[6]
        imdbId = rdata[7]
        ogTitle = rdata[9]
        storyplot = rdata[10]
        popularity = rdata[11]
        posterPath = rdata[12]
        releaseDate = rdata[13]
        revenue = rdata[14]
        runtime = rdata[15]
        tagline = rdata[19]
        collectionId = rdata[20]
        isAdult = bool(rdata[3])

        # converting isAdult into binary representation for storing in database
        if isAdult is False:
            isAdult = 0
        else:
            isAdult = 1
            
        # converting runtime to int object for storing in database
        if runtime != '\\N':
            runtime = int(runtime)
        else:
            runtime = None

        try:
            dtuple = (mvTitle, ogTitle, releaseDate, runtime, isAdult, imdbId, storyplot, popularity, posterPath, collectionId, revenue, budget, tagline, homepage)
            mycursor.execute(tmovie_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("For Index: ",index, "error is: ",exc)

print("Data added to the database successfully!")

In [ ]:
# inserting tmdb person data in tmdbperson table
index = 0
with open('data/tmdb/PERSON.tsv', 'r',encoding='utf8') as f:
    fopen = csv.reader(f, delimiter='\t')

    for rdata in fopen:
        if index == 0:
            index += 1
            continue
        if index >= 50000:
            break
        index += 1
        # Extracting data from the csv file
        id = rdata[1]
        name = rdata[2]
        birthdate = rdata[5]
        alias = rdata[6]
        deathdate = rdata[7]
        gender = rdata[8]
        biography = rdata[9]
        profilePath = rdata[10]
        popularity = rdata[11]
        birthPlace = rdata[12]
        imdbLink = rdata[14]
        homepage = rdata[15]
        homeIndexry = rdata[17]
        #converting birthday and deathday to none if they are not present 
        if birthdate=='' or birthdate=='\\N':
            birthdate=None
        if deathdate=='' or deathdate=='\\N':
            deathday=None
            
        try:
            dtuple = (id, name, birthdate, deathdate, alias, gender, biography, profilePath, popularity, birthPlace, homepage, homeIndexry, imdbLink)
            mycursor.execute(tperson_Query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("For Index: ",index, "error is: ",exc)

print("Data added to the database successfully!")

In [ ]:
# inserting rotten tomatoes movie data in rtmovie table
index = 0
with open('data/all_movie.csv', 'r',encoding='utf8') as f:
    fopen = csv.reader(f, delimiter=',')

    for rdata in fopen:
        if index == 0:
            index += 1
            continue
        if index >= 50000:
            break
        index += 1
        # Extracting data from the csv file
        cast1 = rdata[0]
        cast2 = rdata[1]
        description = rdata[6]
        director = rdata[7]
        genre = rdata[10]
        rating = rdata[11]
        title = rdata[15]
        try:
            dtuple = (title, cast1, cast2, description, director, genre, rating)
            mycursor.execute(rtmovie_query, dtuple)
            mydb.commit()
        except Exception as exc:
            print("For Index: ",index, "error is: ",exc)

print("Data added to the database successfully!")

In [ ]:
# Creating a mapping table using links.csv from movielens dataset which maps movieid to imdbid and tmdbid
index=0
with open('data/movielens/links.csv', 'r',encoding="utf8") as f:
    fopen = csv.reader(f, delimiter=',')

    for rdata in fopen:
        if index == 0:
            index += 1
            continue
        index+=1
        # Extracting data from the csv file
        movieId = rdata[0]
        imdbId = rdata[1]
        tmdbId = rdata[2]

        try:
            dtuple = (movieId, 'tt'+imdbId, tmdbId)
            mycursor.execute(dbmapping_Query, dtuple)
            mydb.commit()
            
        except Exception as exc:
            print("For Index: ",index, "error is: ",exc)

print("Data added to the database successfully!")